In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://unauthenticated-marva-wisely.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://unauthenticated-marva-wisely.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Ming Hsin University of Science and Technology, MUST）是一所位於臺灣新竹縣新豐鄉的私立科技大學，秉持「堅毅、求新、創造」的校訓，以培育「跨域整合、務實創新、全人學習」的專業人才為教育目標，並以成為「一流產業大學」為發展願景。

**歷史沿革**
明新科技大學創立於1966年（民國55年）3月，初為「明新工業專科學校」。1997年（民國86年）7月，經教育部核准改制為「明新技術學院」，並設有專科部。2002年（民國91年）8月，再獲教育部核准升格為「明新科技大學」。2018年12月，更名為「明新學校財團法人明新科技大學」。

**地理位置與資源**
學校地處新竹縣新豐鄉，佔地逾三十公頃，依傍省道縱貫線且毗鄰中山高速公路，交通便利。明新科技大學鄰近新竹科學園區與新竹工業區，擁有豐富的產業資源，與產業界建立多項產學合作計畫，並為區域產業培養大量人才，被譽為新竹縣人才供應的第一品牌。

**學術與特色**
明新科技大學現設有六個學院，包括：半導體學院、工程學院、管理學院、民生學院（亦稱服務產業學院）、人文與設計學院、共同教育學院。學校提供20個學系、2個學位學程（含1個博士學位學程）及11個碩士班。學生總數（含日間部與進修部）約有一萬三千餘人，專任教師約三百八十餘人。

明新科技大學的辦學績效卓著，特別強調與產業鏈結，以「產業大學」為定位。學校掌握半導體、AI、元宇宙、風電綠能等前瞻產業趨勢。在2022年《遠見》雜誌「企業最愛公私立技職科大調查」中，明新科大在四大領域獲得兩項企業最愛，且畢業生起薪在私校中名列第一，甚至優於部分國立科大。根據1111人力銀行統計，半導體產業界最愛聘用的畢業生中，明新科大名列第四，是唯一入榜的私立科大，與台、清、交、成等頂尖大學並列前五大。

學校發展「MUST」四大育才特色，包括：多元學習（Multidisciplinary Learning）、全球視野（Universal Perspective）、永續經營（Sustainable Operations）與技術創新（Technological Innovation），引導學生跨域學習，成為搶手的技職人才。明新科技大學也積極推動國際化，擁有約一千名外籍學生，致力於培養具備國際溝通力的技職菁英。

此外，明新科技大學工學院之土木工

In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長為**呂明峯**教授。他於2025年2月1日正式上任，成為明新科技大學的第11任校長。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
